# TMDB API PULL
Name: Michael McCann <br>
Data: 29 APR 2022

## Imports

In [1]:
import pandas as pd
import numpy as np
import json, os, math, time

import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

## Get API Key, Instantiate API

In [2]:
# Get filepath for keys
file = "../../.secret/tmdb_api.json"

# load keys as login
with open(file, 'r') as f:
    login = json.load(f)

# Instantiate API
tmdb.API_KEY = login['api-key']

## Functions

### Movie Ratings Function

In [26]:
def movie_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
        
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

### Write JSON function

In [33]:
# def write_json(new_data, filename):
#     with open(filename, 'r+') as f:
#         file_data = json.load(f)
        
#         if (type(new_data) == list) & (type(file_data) == list):
#             file_data.extend(new_data)
#         else:
#             file_data.append(new_data)
#         file.seek(0)
        
#         json.dump(file_data, f)
        
def write_json(new_data, filename): 
    with open(filename,'r+') as file:
    # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

## Set Variables

In [34]:
# Created or Confirm folder for data
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

# Read in title basics from IMDB
basics = pd.read_csv('data/title_basics.csv.gz')

# Set Years we are interested in
YEARS_TO_GET = [2000, 2001]

## API Call

In [35]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position = 0):    
    JSON_FILE = f'{FOLDER}tmdb_api_requests_{YEAR}.json'
    
    # Check for JSON_FILE
    file_exists = os.path.isfile(JSON_FILE)

    # If it does not exist create a blank one for use
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    # get movie ids ('tconst') from our IMDB basics dataframe
    df = basics.loc[basics['startYear'] == YEAR].copy()
    movie_ids = df['tconst'].copy()
    
    # Check for previous pulls 
    previous_df = pd.read_json(JSON_FILE)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    for movie_id in tqdm_notebook(movie_ids_to_get,
                             desc=f'Movies from {YEAR}',
                             position = 1,
                             leave = True):
            try:
                temp = movie_rating(movie_id)
                write_json(temp, JSON_FILE)
                time.sleep(0.02)

            except Exception as e:
                    continue
    
    # Save the results out to a zipped csv                
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1397 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1517 [00:00<?, ?it/s]